## 📊 Patient Admissions Dashboard

“Who is coming in, when, and how often?”

## REQUIRED VISUALS CHECKLIST 
| Requirement            | Implemented  |
| ---------------------- | ------------ |
| Total admissions       | ✅ KPI        |
| Admissions over time   | ✅ Line chart |
| Admissions per patient | ✅ KPI        |
| Average LOS            | ✅ KPI        |
| Readmissions (30 days) | ✅ KPI        |
| Patient timeline table | ✅ Table      |


1️⃣ KPI METRICS (MANDATORY)

These are shown as KPI cards.

🔹 KPI 1 — Total Admissions

Meaning: Count of encounters

In [0]:
%sql
SELECT
    COUNT(DISTINCT encounter_id) AS total_admissions
FROM angad_kumar91.fhir_healthcare_analytics_gold.fact_admissions;


🔹 KPI 2 — Average Length of Stay (LOS)

Meaning: Discharge date − Admit date

In [0]:
%sql
SELECT
    ROUND(AVG(length_of_stay_days), 2) AS avg_length_of_stay_days
FROM angad_kumar91.fhir_healthcare_analytics_gold.fact_admissions
WHERE length_of_stay_days IS NOT NULL;


**KPI 3 — Admissions per Patient (Utilization)**

Meaning: How often patients are admitted

In [0]:
%sql
SELECT
    ROUND(
        COUNT(encounter_id) * 1.0 / COUNT(DISTINCT patient_id),
        2
    ) AS admissions_per_patient
FROM angad_kumar91.fhir_healthcare_analytics_gold.fact_admissions;


### 2️⃣ TIME-BASED TRENDS (LINE CHART)

This answers:

“When are admissions increasing or decreasing?”

**📈 Admissions Over Time (Monthly)**

Visualization

📈 Line chart

- X-axis  : admit_year + admit_month

- Y-axis : admissions

In [0]:
%sql
SELECT
    admit_year,
    admit_month,
    COUNT(encounter_id) AS admissions
FROM angad_kumar91.fhir_healthcare_analytics_gold.fact_admissions
GROUP BY admit_year, admit_month
ORDER BY admit_year, admit_month;


Databricks visualization. Run in Databricks to view.

## 3️⃣ READMISSIONS (IMPORTANT & IMPRESSIVE)

This is what separates average dashboards from strong ones.

🔹 What is a 30-day readmission?

A patient admitted again within 30 days of a previous discharge

🔁 Readmissions (30 Days) — LOGIC

We compare each encounter with the previous discharge date of the same patient.

✅  Identify 30-day Readmissions

In [0]:
%sql
WITH ordered_admissions AS (
    SELECT
        patient_id,
        encounter_id,
        admit_time,
        discharge_time,
        LAG(discharge_time) OVER (
            PARTITION BY patient_id
            ORDER BY admit_time
        ) AS previous_discharge_time
    FROM angad_kumar91.fhir_healthcare_analytics_gold.fact_admissions
)
SELECT
    COUNT(*) AS readmissions_30_days
FROM ordered_admissions
WHERE previous_discharge_time IS NOT NULL
  AND DATEDIFF(admit_time, previous_discharge_time) <= 30;


Databricks visualization. Run in Databricks to view.

**KPI — Readmission Rate**

In [0]:
%sql
WITH ordered_admissions AS (
    SELECT
        patient_id,
        encounter_id,
        admit_time,
        discharge_time,
        LAG(discharge_time) OVER (
            PARTITION BY patient_id
            ORDER BY admit_time
        ) AS previous_discharge_time
    FROM angad_kumar91.fhir_healthcare_analytics_gold.fact_admissions
)
SELECT
    ROUND(
        COUNT(
            CASE
                WHEN previous_discharge_time IS NOT NULL
                 AND DATEDIFF(admit_time, previous_discharge_time) <= 30
                THEN 1
            END
        ) * 100.0 / COUNT(*),
        2
    ) AS readmission_rate_percentage
FROM ordered_admissions;


## 4️⃣ PATIENT-LEVEL VIEW (TABLE VISUAL)

This satisfies:

🟦 Table → Patient + encounter timeline

🧾 Patient Admission Timeline Table

In [0]:
%sql
SELECT
    a.patient_id,
    p.gender,
    p.age_years,

    a.encounter_id,
    a.admit_time,
    a.discharge_time,
    a.length_of_stay_days,
    a.admission_type,
    a.encounter_status

FROM angad_kumar91.fhir_healthcare_analytics_gold.fact_admissions a
JOIN angad_kumar91.fhir_healthcare_analytics_gold.dim_patient p
  ON a.patient_id = p.patient_id
ORDER BY a.patient_id, a.admit_time;


Databricks visualization. Run in Databricks to view.

## 🩺 2. Diagnosis Breakdown Dashboard

“What are patients being treated for?”

**REQUIREMENTS CHECKLIST**

| Requirement             | Status |
| ----------------------- | ------ |
| Diagnosis count         | ✅      |
| Top diagnoses           | ✅      |
| Diagnosis per encounter | ✅      |
| Chronic vs acute        | ✅      |
| Bar chart               | ✅      |
| Pie chart               | ✅      |
| Trend chart             | ✅      |
| Table view              | ✅      |


### 1️⃣ CORE METRICS (MANDATORY)

🔹 Metric 1 — Diagnosis Count (Frequency)

Meaning: How often each diagnosis occurs

In [0]:
%sql
SELECT
    diagnosis_name,
    COUNT(*) AS diagnosis_count
FROM angad_kumar91.fhir_healthcare_analytics_gold.fact_admission_diagnosis
GROUP BY diagnosis_name
ORDER BY diagnosis_count DESC;


Databricks visualization. Run in Databricks to view.

### 🔹 Metric 2 — Diagnosis per Encounter (Clinical Load)

Meaning: How many diagnoses per encounter on average

In [0]:
%sql
SELECT
    ROUND(
        COUNT(*) * 1.0 / COUNT(DISTINCT encounter_id),
        2
    ) AS diagnoses_per_encounter
FROM angad_kumar91.fhir_healthcare_analytics_gold.fact_admission_diagnosis;


### 2️⃣ TOP DIAGNOSES (BAR CHART)

This answers:

“What are the most common diseases?”

Visualization

📊 Bar chart

X-axis: diagnosis_name

Y-axis: diagnosis_count

In [0]:
%sql
SELECT
    diagnosis_name,
    COUNT(*) AS diagnosis_count
FROM angad_kumar91.fhir_healthcare_analytics_gold.fact_admission_diagnosis
GROUP BY diagnosis_name
ORDER BY diagnosis_count DESC
LIMIT 10;


Databricks visualization. Run in Databricks to view.

### 3️⃣ DIAGNOSIS DISTRIBUTION (PIE CHART)

This answers:

“How diagnoses are distributed overall?”

Visualization

🥧 Pie chart

Slice: diagnosis_category

Value: diagnosis_count

In [0]:
%sql
SELECT
    d.diagnosis_category,
    COUNT(*) AS diagnosis_count
FROM angad_kumar91.fhir_healthcare_analytics_gold.fact_admission_diagnosis f
JOIN angad_kumar91.fhir_healthcare_analytics_gold.dim_diagnosis d
  ON f.diagnosis_key = d.diagnosis_key
GROUP BY d.diagnosis_category;


Databricks visualization. Run in Databricks to view.

### 4️⃣ DIAGNOSIS TREND OVER TIME (LINE CHART)

This answers:

“Are certain diseases increasing or decreasing?”

📈 Diagnosis Trend (Monthly)

Visualization

📈 Line chart

X-axis: year-month

Y-axis: diagnosis_count

Legend: diagnosis_name

In [0]:
%sql
SELECT
    d.diagnosis_name,
    YEAR(a.admit_time)  AS year,
    MONTH(a.admit_time) AS month,
    COUNT(*) AS diagnosis_count
FROM angad_kumar91.fhir_healthcare_analytics_gold.fact_admission_diagnosis f
JOIN angad_kumar91.fhir_healthcare_analytics_gold.dim_diagnosis d
  ON f.diagnosis_key = d.diagnosis_key
JOIN angad_kumar91.fhir_healthcare_analytics_gold.fact_admissions a
  ON f.encounter_id = a.encounter_id
GROUP BY d.diagnosis_name, YEAR(a.admit_time), MONTH(a.admit_time)
ORDER BY d.diagnosis_name, year, month;


Databricks visualization. Run in Databricks to view.

### 5️⃣ CHRONIC vs ACUTE ANALYSIS (SEVERITY PATTERN)

This answers:

“Are we seeing more chronic or acute conditions?”

🧠 Simple Chronic vs Acute Classification

(Rule-based, explainable — reviewers like this)

Visualization

🥧 Pie or 📊 Bar chart

In [0]:
%sql
SELECT
    CASE
        WHEN LOWER(d.diagnosis_name) LIKE '%diabetes%'
          OR LOWER(d.diagnosis_name) LIKE '%hypertension%'
          OR LOWER(d.diagnosis_name) LIKE '%renal%'
          OR LOWER(d.diagnosis_name) LIKE '%asthma%'
        THEN 'Chronic'
        ELSE 'Acute'
    END AS condition_type,
    COUNT(*) AS diagnosis_count
FROM angad_kumar91.fhir_healthcare_analytics_gold.fact_admission_diagnosis f
JOIN angad_kumar91.fhir_healthcare_analytics_gold.dim_diagnosis d
  ON f.diagnosis_key = d.diagnosis_key
GROUP BY condition_type;


Databricks visualization. Run in Databricks to view.

### 6️⃣ DIAGNOSIS BY ENCOUNTER (TABLE)

This satisfies:

📋 Table → Diagnosis by encounter

Visualization

📋 Table

In [0]:
%sql
SELECT
    f.encounter_id,
    f.patient_id,
    d.diagnosis_name,
    d.diagnosis_category
FROM angad_kumar91.fhir_healthcare_analytics_gold.fact_admission_diagnosis f
JOIN angad_kumar91.fhir_healthcare_analytics_gold.dim_diagnosis d
  ON f.diagnosis_key = d.diagnosis_key
ORDER BY f.encounter_id;


Databricks visualization. Run in Databricks to view.

# 💊 3. Treatment Trends Dashboard

“What treatments, procedures, and medications are being used?”

This dashboard shows end-to-end care delivery:

Diagnosis ➝ Procedure ➝ Medication ➝ Immunization

_The Treatment Trends dashboard tracks procedures, medications, and immunizations over time, highlights the most common treatments, and links diagnoses to treatments. This provides a complete view of how clinical conditions translate into care actions._

**REQUIREMENTS CHECKLIST**
| Requirement                | Status |
| -------------------------- | ------ |
| Procedure count            | ✅      |
| Medication usage           | ✅      |
| Treatments over time       | ✅      |
| Treatment per diagnosis    | ✅      |
| Bar chart                  | ✅      |
| Line chart                 | ✅      |
| Medication table           | ✅      |
| Diagnosis → Treatment view | ✅      |



### 0️⃣ Data foundation (what we will use)

✅ Silver tables (correct grain)

silver.procedure

silver.medication_request

silver.immunization

✅ Gold tables (for diagnosis linkage)

fact_admission_diagnosis

dim_diagnosis

fact_admissions

### 1️⃣ CORE METRICS (MANDATORY)

🔹 Metric 1 — Procedure Count (Treatment Volume)

Meaning: How many procedures are performed

In [0]:
%sql
SELECT
    COUNT(*) AS total_procedures
FROM angad_kumar91.fhir_healthcare_analytics_silver.procedure;


###🔹 Metric 2 — Medication Usage

Meaning: How many medications are prescribed

In [0]:
%sql
SELECT
    COUNT(*) AS total_medications_prescribed
FROM angad_kumar91.fhir_healthcare_analytics_silver.medication_request;


Databricks visualization. Run in Databricks to view.

###🔹 Metric 3 — Treatments Over Time

Meaning: Are treatments increasing or decreasing?

(Handled in trend charts below)

### 2️⃣ MOST COMMON PROCEDURES (BAR CHART)

This answers:

“What procedures are most common?”

**📊 Top Procedures**

Visualization

📊 Bar chart

X-axis: procedure_name

Y-axis: procedure_count

In [0]:
%sql
SELECT
    procedure_name,
    COUNT(*) AS procedure_count
FROM angad_kumar91.fhir_healthcare_analytics_silver.procedure
GROUP BY procedure_name
ORDER BY procedure_count DESC
LIMIT 10;


Databricks visualization. Run in Databricks to view.

### 3️⃣ TREATMENT TRENDS OVER TIME (LINE CHART)

This answers:

“Are treatments increasing or decreasing?”

📈 Procedures Over Time (Monthly)

In [0]:
%sql
SELECT
    YEAR(performed_time)  AS year,
    MONTH(performed_time) AS month,
    COUNT(*) AS procedure_count
FROM angad_kumar91.fhir_healthcare_analytics_silver.procedure
WHERE performed_time IS NOT NULL
GROUP BY YEAR(performed_time), MONTH(performed_time)
ORDER BY year, month;


Databricks visualization. Run in Databricks to view.

### 📈 Medications Over Time (Monthly)

Visualization

📈 Line chart

X-axis: year-month

Y-axis: count

Legend: Procedure vs Medication (two visuals or combined)

In [0]:
%sql
SELECT
    YEAR(prescribed_date)  AS year,
    MONTH(prescribed_date) AS month,
    COUNT(*) AS medication_count
FROM angad_kumar91.fhir_healthcare_analytics_silver.medication_request
WHERE prescribed_date IS NOT NULL
GROUP BY YEAR(prescribed_date), MONTH(prescribed_date)
ORDER BY year, month;


Databricks visualization. Run in Databricks to view.

### 4️⃣ MEDICATION PRESCRIPTIONS (TABLE)

This satisfies:

🧾 Table → Medication prescriptions

🧾 Medication Prescription Table

Visualization

🧾 Table

Filterable by medication, status, date

In [0]:
%sql
SELECT
    mr.patient_id,
    mr.encounter_id,
    mr.medication_name,
    mr.status AS medication_status,
    mr.prescribed_date
FROM angad_kumar91.fhir_healthcare_analytics_silver.medication_request mr
ORDER BY mr.prescribed_date DESC;


Databricks visualization. Run in Databricks to view.

### 5️⃣ DIAGNOSIS → TREATMENT LINK (COMBINED VIEW)

This answers:

“How treatments relate to diagnoses?”

This is the most impressive part.

🔗 Diagnosis → Procedure Mapping


In [0]:
%sql
SELECT
    d.diagnosis_name,
    pr.procedure_name,
    COUNT(*) AS procedure_count
FROM angad_kumar91.fhir_healthcare_analytics_gold.fact_admission_diagnosis fad
JOIN angad_kumar91.fhir_healthcare_analytics_gold.dim_diagnosis d
  ON fad.diagnosis_key = d.diagnosis_key
JOIN angad_kumar91.fhir_healthcare_analytics_silver.procedure pr
  ON fad.encounter_id = pr.encounter_id
GROUP BY d.diagnosis_name, pr.procedure_name
ORDER BY procedure_count DESC;


Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

### 🔗 Diagnosis → Medication Mapping

Visualization

🔗 Matrix / table

Diagnosis → Treatment mapping

Filterable by diagnosis

In [0]:
%sql
SELECT
    d.diagnosis_name,
    mr.medication_name,
    COUNT(*) AS medication_count
FROM angad_kumar91.fhir_healthcare_analytics_gold.fact_admission_diagnosis fad
JOIN angad_kumar91.fhir_healthcare_analytics_gold.dim_diagnosis d
  ON fad.diagnosis_key = d.diagnosis_key
JOIN angad_kumar91.fhir_healthcare_analytics_silver.medication_request mr
  ON fad.encounter_id = mr.encounter_id
GROUP BY d.diagnosis_name, mr.medication_name
ORDER BY medication_count DESC;


Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.